In [1]:
import numpy as np
from tqdm import tqdm
from scipy import stats

In [2]:
class Dynamics:
    pass

## Problem 1

In [3]:
hw6p1dynamics=Dynamics()
hw6p1dynamics.S0 = np.array([100,110])
hw6p1dynamics.r = 0.05
hw6p1dynamics.correlations = np.array([[1.,0.8],[0.8,1.]])  #You fill this in with a 2x2 correlation matrix
hw6p1dynamics.sigma = np.diag([0.3, 0.2])

In [4]:
class Contract:
    pass

In [5]:
hw6p1contract=Contract() 
hw6p1contract.K = 110 
hw6p1contract.T = 1.0 
hw6p1contract.weights = np.array([1/2, 1/2])

In [6]:
class MC:
    pass

In [7]:
hw6p1MC=MC()
hw6p1MC.M = 10000  # Number of paths.  
hw6p1MC.seed = 0  # Seeding the random number generator with a specified number helps make the calculations reproducible

In [8]:
def BS(sigma,S,r,R,K,T):
    F = S*np.exp(R*T)
    sd = sigma*np.sqrt(T)
    if sd==0:
        return np.maximum(np.exp(-r*T)*(F-K),0)
    d1 = np.log(F/K)/sd+sd/2
    d2 = np.log(F/K)/sd-sd/2
    return np.exp(-r*T)*(F*stats.norm.cdf(d1)-K*stats.norm.cdf(d2))

def pricer_callonbasket_GBM_MC(contract,dynamics,MC):

    np.random.seed(MC.seed)  #seed the random number generator
    
    S0, r, corr, sigma = dynamics.S0, dynamics.r, dynamics.correlations, dynamics.sigma
    K, T, weights = contract.K, contract.T, contract.weights
    M = MC.M
    
    if MC.antithetic is True and MC.control is False:
        
        C = np.zeros(2*M)
        
        for i in tqdm(range(M)):
            
            cov = T* (sigma @ corr @ sigma.T)
            Z1 = np.random.multivariate_normal(np.zeros(len(S0)), cov)
            Z2 = -Z1
            S1 = np.zeros(len(Z1))
            S2 = np.zeros(len(Z2))
            for j, z in enumerate(Z1):
                S1[j] = S0[j]*np.exp((r-(sigma[j][j]**2)/2)*T+z)
            for j, z in enumerate(Z2):
                S2[j] = S0[j]*np.exp((r-(sigma[j][j]**2)/2)*T+z)    
                
            H1 = weights @ S1
            H2 = weights @ S2
            
            C[2*i] = np.exp(-r*T)*np.maximum(H1-K,0)
            C[2*i+1] = np.exp(-r*T)*np.maximum(H2-K,0)
            
        call_price = np.mean(C)
        standard_error = np.std(C)/np.sqrt(len(C))
    
    elif MC.antithetic is False and MC.control is True:
        
        Y = np.zeros(M)
        Y_star = np.zeros(M)
        
        for i in tqdm(range(M)):
            
            cov = T* (sigma @ corr @ sigma.T)
            Z = np.random.multivariate_normal(np.zeros(len(S0)), cov)
            S = np.zeros(len(Z))
            for j, z in enumerate(Z):
                S[j] = S0[j]*np.exp((r-(sigma[j][j]**2)/2)*T+z)
                
            H = weights @ S
            G = np.prod(S)**(1/len(S))
            Y[i] = np.exp(-r*T)*np.maximum(H-K,0)
            Y_star[i] = np.exp(-r*T)*np.maximum(G-K,0)
        
        cov = np.cov(Y,Y_star)
        beta = cov[0][1]/cov[1][1] # Covariance / Variance
        
        S0_star = np.prod(S0)**(1/len(S0))
        R_star = r + (1/8)*(2*corr[0][1]*sigma[0][0]*sigma[1][1]-sigma[0][0]**2-sigma[1][1]**2)
        sigma_star = np.sqrt((1/4)*(sigma[0][0]**2+2*corr[0][1]*sigma[1][1]*sigma[0][0]+sigma[1][1]**2))
        
        YM = np.mean(Y)
        YM_star = np.mean(Y_star)
        C_star = BS(sigma_star,S0_star,r,R_star,K,T)
        
        Y_CV =  Y + beta*(C_star - Y_star)
        
        call_price = np.mean(Y_CV)
        standard_error = np.std(Y_CV)/np.sqrt(len(Y_CV))
    
    elif MC.antithetic is False and MC.control is False:
        
        C = np.zeros(M)
        cov = T* (sigma @ corr @ sigma.T)
        
        for i in tqdm(range(M)):
            
            Z = np.random.multivariate_normal(np.zeros(len(S0)), cov)
            S = np.zeros(len(Z))
            for j, z in enumerate(Z):
                S[j] = S0[j]*np.exp((r-(sigma[j][j]**2)/2)*T+z)
                
            H = weights @ S
            C[i] = np.exp(-r*T)*np.maximum(H-K,0)
            
        call_price = np.mean(C)
        standard_error = np.std(C)/np.sqrt(len(C))
                
            
    
    # You complete the coding of this function
    # You are not required to support the case where MC.control = MC.antithetic = True
    # (simultaneous use of control variate and antithetic)
    # But you are required to support the other 3 possible settings of MC.antithetic/MC.control 
    # namely False/False, True/False, False/True.
    # (ordinary MC, antithetic without control, control without antithetic)
        
    return(call_price, standard_error)
        

In [9]:
hw6p1MC.antithetic = False  
hw6p1MC.control = False 
(call_price_ordinary, std_err_ordinary) = pricer_callonbasket_GBM_MC(hw6p1contract,hw6p1dynamics,hw6p1MC)
print(call_price_ordinary, std_err_ordinary)

100%|██████████████████████████████████████████████████████████████████████████| 10000/10000 [00:01<00:00, 8950.72it/s]

10.028517056790681 0.1662128969435554


In [10]:
hw6p1MC.antithetic = True  
hw6p1MC.control = False 
(call_price_AV, std_err_AV) = pricer_callonbasket_GBM_MC(hw6p1contract,hw6p1dynamics,hw6p1MC)
print(call_price_AV, std_err_AV)

100%|██████████████████████████████████████████████████████████████████████████| 10000/10000 [00:01<00:00, 6980.27it/s]

9.905530841296681 0.11814261061406257


In [11]:
hw6p1MC.antithetic = False
hw6p1MC.control = True 
(call_price_CV, std_err_CV) = pricer_callonbasket_GBM_MC(hw6p1contract,hw6p1dynamics,hw6p1MC)
print(call_price_CV, std_err_CV)

100%|██████████████████████████████████████████████████████████████████████████| 10000/10000 [00:01<00:00, 7424.87it/s]

9.98526430014317 0.004439750432672437


## Problem 2

In [12]:
hw6p2dynamics=Dynamics()
hw6p2dynamics.sigma = 0.2
hw6p2dynamics.S0 = 100
hw6p2dynamics.r = 0.02

In [13]:
hw6p2contract=Contract()
hw6p2contract.K = 150
hw6p2contract.T = 1

In [14]:
hw6p2MC=MC()
hw6p2MC.M = 100000  # Number of paths
hw6p2MC.seed = 0  # Seeding the random number generator with a specified number helps make the calculations reproducible

In [15]:
def pricer_call_GBM_MCwithdriftchange(contract,dynamics,MC):

    np.random.seed(MC.seed)  #seed the random number generator
    
    sigma, S0, r = dynamics.sigma, dynamics.S0, dynamics.r
    K, T = contract.K, contract.T
    M, lamb = MC.M, MC.lamb
    
    Y = np.zeros(M)

    for i in tqdm(range(M)):
        Z = np.random.normal()
        S = S0*np.exp((r+lamb*sigma-(sigma**2)/2)*T+sigma*np.sqrt(T)*Z)
        Y[i] = np.exp(-r*T)*np.maximum(S-K,0)
        
        if abs(lamb - 0) > 1e-12:
            IS = np.exp(-lamb*np.sqrt(T)*Z-0.5*T*lamb**2)
            Y[i] = IS*Y[i]
    
    call_price = np.mean(Y)
    standard_error = np.std(Y)/np.sqrt(len(Y))
    # You complete the coding of this function
        
    return(call_price, standard_error)
        

In [16]:
hw6p2MC.lamb = 0  # Zero drift adjustment gives ordinary MC
(call_price_ordinary, std_err_ordinary) =  pricer_call_GBM_MCwithdriftchange(hw6p2contract,hw6p2dynamics,hw6p2MC)
print(call_price_ordinary, std_err_ordinary)

100%|███████████████████████████████████████████████████████████████████████| 100000/100000 [00:01<00:00, 89407.06it/s]

0.24852756686116262 0.007435819033778785


In [17]:
hw6p2MC.lamb =  (1/hw6p2dynamics.sigma)*(np.log(hw6p2contract.K*1.1/hw6p2dynamics.S0)/hw6p2contract.T-hw6p2dynamics.r)
(call_price_importsamp, std_err_importsamp) =  pricer_call_GBM_MCwithdriftchange(hw6p2contract,hw6p2dynamics,hw6p2MC)
print(call_price_importsamp, std_err_importsamp)

100%|███████████████████████████████████████████████████████████████████████| 100000/100000 [00:01<00:00, 62592.88it/s]

0.24989864051776664 0.0007761247577340426


In [18]:
hw6p2MC.lamb

2.4038764395624455